# Problem 1
### Elliptic Curve Arithmetic
1) Write three programs for computing the negative of a point, the sum of two points and the double of a point, respectively.

2) A scalar multiplication of a point P by an integer k, denoted kP or [k]P, is the sum

In [1]:
import sage.rings.finite_rings.integer_mod_ring as zp

In [29]:
class ECPoint:
    
    def __init__(self, ec, x, y, infinity=False):
        self.ec = ec
        self.x = x
        self.y = y
        self.infinity= infinity

    def __str__(self):
        if self.infinity:
            return f'ECPoint(Infinity)'
        return f'ECPoint({self.x}, {self.y})'
    
    def __repr__(self):
        return self.__str__()
    
    
    def negative(self):
        if self.infinity:
            return self
        return ECPoint(self.ec, self.x, -self.y)
    
    
    def __eq__(self, ecp):
        return self.x == ecp.x and self.y == ecp.y
    
    def __add__(self, ecpoint):
        assert self.ec == ecpoint.ec
        if self.infinity:
            return ecpoint
        elif ecpoint.infinity:
            return self
        if self.x == ecpoint.x and self.y == -ecpoint.y:
            return self.ec.infinity_point()
        if self != ecpoint:
            lam = (ecpoint.y - self.y) / (ecpoint.x - self.x)
        else:
            lam = (3 * self.x**2 + self.ec.a) / (2 * self.y)
        x3 = lam ** 2 - self.x - ecpoint.x
        return ECPoint(self.ec, x3, lam * (self.x - x3) - self.y)
    
    def __sub__(self, ecp):
        return self + ecp.negative()
    
    def double(self):
        return self + self
    
    def scalar_mult(self, k):
        if k < 0:
            return self.negative().scalar_mult(abs(k))
        elif k == 0:
            return self.ec.infinity_point()
        chrs = "{0:b}".format(k)[1:]
        res = self
        for c in chrs:
            if c == '0':
                res = res.double()
            else:
                res = res.double() + self
        return res
        
class EC:

    def __init__(self,p, a, b, order=None):
        assert 4 * a**3 + 27 * b**2 != 0
        self.p = zp.IntegerModRing(p)
        self.a = self.p(a)
        self.b = self.p(b)
        if order ==None:
            self.order = EllipticCurve(self.p, [self.a, self.b]).order()
        else:
            self.order = order
        
    def liftX(self, x):
        return ECPoint(self, x, sqrt(x**3 + self.a *x + self.b))
    
    def get_elem(self, x, y):
        if y ** 2 != x**3 + self.a *x + self.b:
            raise ValueError
        return ECPoint(self, x, y)
    
    def infinity_point(self):
        return ECPoint(self, None, None, True)
    
    def __max_t(self):
        return abs(int(sqrt(int(self.p.order()), prec=10)))

In [3]:
ecfield = EC(19, 1, 3)
p6 = ecfield.liftX(6)
(ecfield.liftX(6) + ecfield.infinity_point()) == ecfield.liftX(6)

True

In [4]:
p6 == ecfield.liftX(6)

True

In [5]:
p6.double()

ECPoint(16, 7)

In [6]:
p6 - ecfield.liftX(6)

ECPoint(Infinity)

In [7]:
p6 + ecfield.liftX(1)

ECPoint(13, 3)

In [8]:
p6.scalar_mult(2) == p6.double()

True

In [9]:
p6.double() + p6 == p6.scalar_mult(3)

True

In [10]:
p6.negative().double() == p6.scalar_mult(-2)

True

# Problem 2
### Elliptic Curve Discrete Logarithm Problem (ECDLP)

1) Write a program that solves the ECDLP in cyclic subgroups of E(Fp) using the
Baby Step – Giant Step method.

In [102]:
sqrtrem = lambda x: Integer(x).sqrtrem()
sqrt_up = lambda x: sqrtrem(x)[0] + int(sqrtrem(x)[1] > 0)
def shanks_ec(ring, g, res):
    def get_x(i, j):
        return i*s - j
    s = sqrt_up(ring.order)
    bs = [(res + g.scalar_mult(j), j)for j in range(0, s+1)]
    gs = [(g.scalar_mult(i*s), i) for i in range(0, s+1)]
    reslist = []
    for bi in bs:
        for gi in gs:
            if bi[0] == gi[0]:
                reslist.append([gi[1], bi[1]])
    return list(map(lambda x: get_x(x[0], x[1]), reslist))[0]
ecu = EC(p=19, a=2, b=2)
elem = ecu.liftX(3)
res = elem.scalar_mult(5)
shanks_ec(ecu, elem, res) == 5

True

In [106]:
p = 311
ecu = EC(p=p, a=5, b=-9, order=103)
gen = ecu.get_elem(23, 12)
res = ecu.get_elem(254, 231)
x = shanks_ec(ecu, gen, res)
print("x:", x)
print(f'{gen} ^ {x} =',gen.scalar_mult(x))

x: 74
ECPoint(23, 12) ^ 74 = ECPoint(254, 231)
